In [ ]:
#fakefriends.csv

user_id: 사용자 ID name: 사용자 이름 age: 사용자 나이 num_friends: 친구 수

In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_04_RDDEXAM")
spark =SparkContext(conf=conf).getOrCreate()

24/12/05 16:15:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 16:15:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/05 16:15:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import os
directory= os.path.join(os.getcwd(), "data")
filename= "fakefriends.csv"
filepath= os.path.join(directory, filename)

#RDD생성 
lines= spark.textFile("file://"+filepath.replace("\\", "/"))
lines

file:///home/lab10/src/data/fakefriends.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
def parse(row):
    fields=row.split(",")
    user_id= fields[0]
    id_name= fields[1]
    age= int(fields[2])
    num_friends= int(fields[3])
    return user_id,id_name,age,num_friends  #변수 이름 공백 x 

In [3]:
lines.take(5)

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [4]:
 #1. 평균 친구수 구하기

average_friends= (lines.map(lambda row: int(row.split(",")[3])).mean())

In [5]:
average_friends

248.53200000000015

In [ ]:
#2. 가장 친구가 많은 사용자 찾기

In [6]:
#1
#각 사용자에 대해 user_id와 친구 수를 튜플로 저장
user_fr= lines.map(lambda row: (row.split(",")[1], int(row.split(",")[3])))

In [7]:
most_f= user_fr.reduce(lambda a,b: a if a[1]> b[1] else b)

In [9]:
print(f"가장 친구가 많은 사용자: {most_f[0]} ({most_f[1]}명)")

가장 친구가 많은 사용자: Julian (499명)


In [ ]:
# 3.친구 수가 100 이상인 사용자 필터링

In [14]:
parsed_data = lines.map(parse)

In [17]:
parsed_data.take(5)

[('0', 'Will', 33, 385),
 ('1', 'Jean-Luc', 26, 2),
 ('2', 'Hugh', 55, 221),
 ('3', 'Deanna', 40, 465),
 ('4', 'Quark', 68, 21)]

In [18]:
filtered_user= parsed_data.filter(lambda x: x[3]>=100)

In [19]:
filtered_user.take(5)

[('0', 'Will', 33, 385),
 ('2', 'Hugh', 55, 221),
 ('3', 'Deanna', 40, 465),
 ('5', 'Weyoun', 59, 318),
 ('6', 'Gowron', 37, 220)]

In [20]:
filtered_user_list=filtered_user.collect()

In [25]:
for user in filtered_user_list:
    print(f"사용자: {user[1]}, 친구수: {user[3]}")

사용자: Will, 친구수: 385
사용자: Hugh, 친구수: 221
사용자: Deanna, 친구수: 465
사용자: Weyoun, 친구수: 318
사용자: Gowron, 친구수: 220
사용자: Will, 친구수: 307
사용자: Jadzia, 친구수: 380
사용자: Hugh, 친구수: 181
사용자: Odo, 친구수: 191
사용자: Ben, 친구수: 372
사용자: Keiko, 친구수: 253
사용자: Jean-Luc, 친구수: 444
사용자: Weyoun, 친구수: 323
사용자: Jean-Luc, 친구수: 455
사용자: Geordi, 친구수: 246
사용자: Odo, 친구수: 220
사용자: Miles, 친구수: 268
사용자: Keiko, 친구수: 271
사용자: Ben, 친구수: 445
사용자: Julian, 친구수: 100
사용자: Leeta, 친구수: 363
사용자: Martok, 친구수: 476
사용자: Nog, 친구수: 364
사용자: Keiko, 친구수: 175
사용자: Miles, 친구수: 161
사용자: Nog, 친구수: 281
사용자: Dukat, 친구수: 197
사용자: Jean-Luc, 친구수: 249
사용자: Beverly, 친구수: 305
사용자: Morn, 친구수: 192
사용자: Odo, 친구수: 413
사용자: Hugh, 친구수: 167
사용자: Guinan, 친구수: 345
사용자: Nerys, 친구수: 244
사용자: Ezri, 친구수: 254
사용자: Quark, 친구수: 283
사용자: Lwaxana, 친구수: 212
사용자: Beverly, 친구수: 269
사용자: Keiko, 친구수: 278
사용자: Gowron, 친구수: 194
사용자: Odo, 친구수: 294
사용자: Hugh, 친구수: 158
사용자: Morn, 친구수: 284
사용자: Geordi, 친구수: 100
사용자: Kasidy, 친구수: 442
사용자: Guinan, 친구수: 169
사용자: Geordi, 친구수: 477
사용자: Jadz

In [ ]:
# 특정 나이에 속한 사람들의 친구 수 합계

In [34]:
def get_age_group(age):
    if 10 <= age <= 19:
        return "10대"
    elif 20 <= age <= 29:
        return "20대"
    elif 30 <= age <= 39:
        return "30대"
    elif 40 <= age <= 49:
        return "40대"
    elif 50 <= age <= 59:
        return "50대"
    elif 60 <= age <= 69:
        return "60대"
    else:
        return "기타"

In [35]:
age_groups_rdd= parsed_data.map(lambda x:(get_age_group(x[2]),x[3]))

In [36]:
age_groups_rdd.take(5)

[('30대', 385), ('20대', 2), ('50대', 221), ('40대', 465), ('60대', 21)]

In [37]:
age_group_sum= age_groups_rdd.groupByKey().mapValues(lambda friends: sum(friends))

In [38]:
for group, total_friends in age_group_sum.collect():
    print(f"{group} 친구 수 합계: {total_friends}")
    

30대 친구 수 합계: 22887
20대 친구 수 합계: 21503
50대 친구 수 합계: 24360
40대 친구 수 합계: 26455
60대 친구 수 합계: 23968
10대 친구 수 합계: 5093
